# Generating Qubit Hamiltonians

In [1]:
from utility import *
import numpy as np 

Specify the Qubit Hamiltonian of a molecule by its name, internuclear distances, basis set, and fermion-to-qubit transformation.
Here, we show the resulting Hamiltonian for $H_2$ in STO-3G with $1\overset{\circ}{A}$ between the $H$ atoms. 

In [2]:
qubit_transf = 'jw' # Jordan-Wigner transformations
h2 = get_qubit_hamiltonian(mol='h2', geometry=1, basis='sto3g', qubit_transf=qubit_transf)
print("H:\n{}".format(h2))

H:
-0.32760818967480887 [] +
-0.04919764587136755 [X0 X1 Y2 Y3] +
0.04919764587136755 [X0 Y1 Y2 X3] +
0.04919764587136755 [Y0 X1 X2 Y3] +
-0.04919764587136755 [Y0 Y1 X2 X3] +
0.13716572937099497 [Z0] +
0.15660062488237947 [Z0 Z1] +
0.10622904490856075 [Z0 Z2] +
0.15542669077992832 [Z0 Z3] +
0.13716572937099492 [Z1] +
0.15542669077992832 [Z1 Z2] +
0.10622904490856075 [Z1 Z3] +
-0.13036292057109117 [Z2] +
0.16326768673564346 [Z2 Z3] +
-0.13036292057109117 [Z3]


This Qubit Hamiltonian encodes all of the $2^N$ eigenstates and eigenenergies, where $N=4$ is the number of qubits. 
In principle, one can represent the Hamiltonian as a matrix with $2^N$ dimensions and find the eigenvalues classically through diagonalization, but the cost of this approach clearly grows exponentially with $N$. 

Still, we can check that this approach indeed recovers the ground state energy against method in S1. 

In [3]:
# Defining pauli matrices 
I, X, Y, Z = np.identity(2), np.array([[0, 1], [1, 0]]), np.array([[0, -1j], [1j, 0]]), np.array([[1, 0], [0, -1]])

# Build matrix representiation of the Hamiltonian H
n_qubits = openfermion.count_qubits(h2)
h2_matrix = np.zeros((2**n_qubits, 2**n_qubits), dtype=np.complex)
for term, term_coeff in h2.terms.items(): # Iterate over pauli-words of H
    term = dict(term) # Dict[qubit_index, 'X'/'Y'/'Z']
    
    # Build matrix rep of current pauli-word using kronecker product to represent x_i y_j ...
    pw_matrix = np.identity(1)
    for i in range(n_qubits):
        if i not in term:        pw_matrix = np.kron(pw_matrix, I)
        else:
            if term[i] == 'X':   pw_matrix = np.kron(pw_matrix, X)
            elif term[i] == 'Y': pw_matrix = np.kron(pw_matrix, Y)
            else:                pw_matrix = np.kron(pw_matrix, Z)
    h2_matrix += pw_matrix * term_coeff

eigvals, _ = np.linalg.eigh(h2_matrix)
print("The ground state energy from S1: ")
obtain_PES('h2', [1], 'sto-3g', 'fci')
print("\nThe eigenvalues in the matrix representation of Hamiltonian: \n{}".format(eigvals))

The ground state energy from S1: 
E = -1.1011503301329564 Eh

The eigenvalues in the matrix representation of Hamiltonian: 
[-1.10115033 -0.74587179 -0.74587179 -0.74587179 -0.60860671 -0.60860671
 -0.58166697 -0.58166697 -0.35229063 -0.06021529 -0.06021529 -0.05994379
 -0.05994379  0.03904763  0.50196598  0.52917721]


Alternatively, the qubit-tapering technique can find a smaller effective Hamitlonian by subsitituting operators with $\pm 1$. This technique is detailed in Bravyi's work ([Bravyi et al., "Tapering off qubits to simulate fermionic Hamiltonians", arXiv:1701.08213](https://arxiv.org/abs/1701.08213)). 

In [4]:
print("The effective Hamiltonian:\n {}".format(taper_hamiltonian(h2, n_spin_orbitals=4, n_electrons=2, qubit_transf=qubit_transf))) 

The effective Hamiltonian:
 -0.5310513494337641 [] +
0.1967905834854702 [X0] +
-0.5350572998841723 [Z0]


We can verify that this new Hamiltonian still encodes the ground state energy. 

In [5]:
# Building the matrix representation of the effective Hamiltonian
h2_matrix = -0.53105134 * I + 0.19679058 * X - 0.53505729 * Z

# Obtain the eigenvalues
eigvals, _ = np.linalg.eigh(h2_matrix)
print("The eigenvalues in the effective Hamiltonian: \n {}".format(eigvals))

The eigenvalues in the effective Hamiltonian: 
 [-1.10115031  0.03904763]


## Question 1.

Q) What are the requirements for a function of qubit operators to be a valid mapping for the
fermionic operators?

A) Let say, we have a fock space $ \mathcal{H}_{tgt}\equiv(\mathbb{C}^2)^{\otimes N_f} $ and its corresponding fermion operators are $\{ \hat{a}_{i} \} _{i=1}^{N_f}$ there is a target hamiltonian to be simulated

$$ H_{tgt} = \sum_{i,j}{h_{ij}\hat{a}_i^{\dagger}\hat{a}_j} + \sum_{ijkl}{g_{ijkl} \hat{a}_i^{\dagger}\hat{a}_j^{\dagger}\hat{a}_k\hat{a}_l} $$.

, where the anti-commutation law holds for fermion operators.

$$\{\hat{a}_i, \hat{a}_j\} = 0 \\ \{\hat{a}_i^\dagger, \hat{a}_j\} = \delta_{ij} $$

Also, there is a simulation system with $Q$ qubits, $ \mathcal{H}_{sim}\equiv(\mathbb{C^2})^{\otimes Q} $ .

Then we describe the transformation as an isometry connecting them,

$\mathcal{E} : \mathcal{H}_{tgt} \rightarrow \mathcal{H}_{sim}$.

Therefore, any state of the target system, $|\phi\rangle \in \mathcal{H}_{tgt} $ is identified with a state
$ \mathcal{E}|\phi\rangle \in \mathcal{H}_{sim}$, and the hamiltonian $H_{tgt}$ is mapped to $H_{sim}$ as

$$ H_{sim} = \mathcal{E} H_{tgt} \mathcal{E}^{\dagger} $$.

Then the requirements for $\mathcal{E}$ is as below.
1. It should be unitary equivalent to $H_{sim}$ to preserve the eigenvalues of $H_{tgt}$.
2. (optional) The codespace of $\mathcal{H}_{sim}$ should correspond to the codespace of $\mathcal{H}_{tgt}$,
with some constraints i.e. the number of electrons or spin.

For a simple example, in Jordan-Wigner transform, $\mathcal{E}=I$ and $Q=N_f$, so that the codespace of $\mathcal{H}_{sim}$ is
equivalent to that of $\mathcal{H}_{tgt}$.

The next one, let's consider parity transform with tapered qubits due to the fixed number of electrons, and singlet constraint.
For a bitstring $x$, and $|x\rangle \in \mathcal{H}_{tgt}$,

$ \mathcal{E}|x\rangle = |Ax\rangle$,

where,

$ A=\begin{pmatrix}
1 & 0 & 0 &\cdots & 0\\
1 & 1 & 0 &\cdots & 0\\
1 & 1 & 1 &\cdots & 0\\
\vdots & \vdots & \vdots & \ddots &\vdots\\
1 & 1 & 1 &\cdots & 1
\end{pmatrix}$

and $ Q = N_{f} $.

Then, we order the fermion operators to be the first half correspond to down-spin orbitals, and the rest to up-spin orbitals.


$ \hat{a}_{1\downarrow}, \hat{a}_{2\downarrow}, \cdots \hat{a}_{N_f/2\downarrow}, \hat{a}_{1\uparrow}, \cdots \hat{a}_{N_f/2\uparrow}$
$ = \hat{a}_1, ... \hat{a}_{N_f}$


Then the particle operators for up and down spin should be

$ \hat{N}_{\downarrow}=\sum_{i=1}^{N_f/2}{\hat{a}_i^{\dagger}\hat{a}_i} \quad \hat{N}_{\uparrow}=\sum_{i=N_f/2+1}^{N_f}{\hat{a}_i^{\dagger}\hat{a}_i} $.

and the bit values at the middle and final site of the code in simulator system are

$$(Ax)_{N_f/2}=\sum_{i=1}^{N_f/2}x_i\quad$$
$$(Ax)_{N_f}=\sum_{i=1}^{N_f}x_i$$


Comparing these two sets of equations, we know

$$\mathcal{E}(-1)^{\hat{N}_{\downarrow}}\mathcal{E}^{\dagger}=Z_{N_f/2} \quad \mathcal{E}(-1)^{\hat{N}_{\downarrow}+\hat{N}_{\uparrow}}\mathcal{E}^{\dagger}=Z_{N_f}$$

and since non-relativistic hamiltonian conserves the number of particles with a fixed spin orientation,

$$ [H_{tgt}, \hat{N}_{\downarrow}] = [H_{tgt}, \hat{N}_{\uparrow}] = 0 $$.

Concluding the above all, $H_{sim}$ shares the eigenvectors with $Z_{N_f}$ and $Z_{N_f/2}$.
Therefore, we just replace $N_f$<sup>th</sup> and $N_f/2$<sup>th</sup> to simply $\pm1$.
and the final reduced parity mapping $\mathcal{E}_{r}$ maps to $N_f -2$ qubit simulator system.


## Question 2.

Q) The electronic Hamiltonian is real (due to time-reversal symmetry), what consequences does
that have on the terms in the qubit Hamiltonian after the Jordan-Wigner transformation?

A) The fermion hamiltonian is given as:

$$ H_{tgt} = \sum_{i,j}{h_{ij}\hat{a}_i^{\dagger}\hat{a}_j} + \sum_{ijkl}{g_{ijkl} \hat{a}_i^{\dagger}\hat{a}_j^{\dagger}\hat{a}_k\hat{a}_l}$$.

Its coefficients are real, and since the hamiltonian is hermitian,

$h_{ij} = h_{ji} \quad g_{ijkl}=g_{lkji}$


So let's rewrite the hamiltonian as:

$$ H_{tgt} = \sum_{i}{h_{ii}\hat{a}^{\dagger}_i\hat{a}_i}+\sum_{i<j}{h_{ij}(\hat{a}_i^{\dagger}\hat{a}_j+\hat{a}_j^{\dagger}\hat{a}_i)}$$

$$+ \sum_{i>l \cup j>k}{g_{ijkl} (\hat{a}_i^{\dagger}\hat{a}_j^{\dagger}\hat{a}_k\hat{a}_l+\hat{a}_l^{\dagger}\hat{a}_k^{\dagger}\hat{a}_j\hat{a}_i})$$

$$+ \sum_{i,j}{g_{ijji} \hat{a}_i^{\dagger}\hat{a}_j^{\dagger}\hat{a}_j\hat{a}_i} $$.

We call the terms in each summation as one-body number operators, one-body excitation operators, two-body excitation operators,
and two-body number operators.

And, it is transformed to a qubit hamiltonian, $H_{sim}$, through the Jordan-Wigner transformation:

$\begin{cases} \hat{a}_k \rightarrow \hat{Z}_{k{\leftarrow}}\hat{q}_k\\
\hat{a}_k^{\dagger} \rightarrow \hat{Z}_{k{\leftarrow}}\hat{q}_k^{\dagger} \end{cases}$


,where $\hat{Z}_{k\leftarrow}$ means action of the pauli-Z operator on the qubits with indices less than $k$,
and $\hat{q}_k^{(\dagger)}$ is the qubit annihilation (creation) operator on the qubit $k$. So we can check
the codes of two systems are equivalent while the anti commutation law holds.

The simulator hamiltonian becomes a sum of pauli words, shown as below.

1. One-body number operators

    $\hat{a}_i^{\dagger}\hat{a}_i \rightarrow \hat{q}_i^{\dagger}\hat{q}_i=\frac{1}{2}(\hat{I}-\hat{Z}_i)$

2. One-body excitation operators

    $\hat{a}_i^{\dagger}\hat{a}_j + \hat{a}_j^{\dagger}\hat{a}_i \rightarrow \hat{Z}_{(i \sim j)}(\hat{q}_i^{\dagger}\hat{q}_j + \hat{q}_j^{\dagger}\hat{q}_i) $
    $=\frac{1}{2}\hat{Z}_{(i \sim j)}(\hat{X}_i\hat{X}_j+\hat{Y}_i\hat{Y}_j)$

3. Two-body number operators

    $\hat{a}_i^{\dagger}\hat{a}_j^{\dagger}\hat{a}_j\hat{a}_i \rightarrow \frac{1}{4}(\hat{I}-\hat{Z}_i-\hat{Z}_j+\hat{Z}_i\hat{Z}_j)$

4. Two-body excitation operators

    $\hat{a}_i^{\dagger}\hat{a}_j^{\dagger}\hat{a}_k\hat{a}_l+\hat{a}_l^{\dagger}\hat{a}_k^{\dagger}\hat{a}_j\hat{a}_i\rightarrow$

    $  \pm \hat{Z}_{l{\leftarrow}}\hat{Z}_{k{\leftarrow}}\hat{Z}_{j{\leftarrow}}\hat{Z}_{i{\leftarrow}}(\hat{q}_i^{\dagger}\hat{q}_j^{\dagger}\hat{q}_k\hat{q}_l$
    $  + \hat{q}_i\hat{q}_j\hat{q}_k^{\dagger}\hat{q}_l^{\dagger}$)

Therefore, the coefficient of each pauli word in the qubit hamiltonian is real.
